# Train mBART for RUSSE Detox 2022

## Proprocess data

In [ ]:
!git clone https://github.com/skoltech-nlp/russe_detox_2022

Cloning into 'russe_detox_2022'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 82 (delta 22), reused 43 (delta 16), pack-reused 29
Unpacking objects: 100% (82/82), done.


In [ ]:
import re
import pandas as pd

# пока что мы объединяем dev и train: не хотим валидироваться на обучении, раз существует test-сет
data_df = pd.concat([
           pd.read_csv("./russe_detox_2022/data/input/dev.tsv", sep="\t"),
           pd.read_csv("./russe_detox_2022/data/input/train.tsv", sep="\t").drop(["index"], axis=1)
], axis=0).reset_index(drop=True)

# если у фразы несколько вариантов исправления — просто кладём их все как пары input-target
# исходим из того, что наши модели достаточно complex, чтобы это их не запутало
train_dict = {
    "input": [],
    "target": []
}

for tc, nc1, nc2, nc3 in zip(list(data_df["toxic_comment"]), list(data_df["neutral_comment1"]),
                             list(data_df["neutral_comment2"]), list(data_df["neutral_comment3"])):
  # здесь немножко препроцесса: из-за особенностей sentencepiece-токенизации модели плохо понимают
  # текст, написанный КАПСОМ. мы будем смотреть, если во входной строке больше 50% символов — капс,
  # и в таком случае приводить её к нижнему регистру
  input_str = str(tc)
  if len([c for c in input_str if re.search(r"[A-ZА-ЯЁ]", c)]) / len(input_str) > 0.5:
    input_str = input_str.lower()

  train_dict["input"].append(input_str)
  train_dict["target"].append(str(nc1))
  if type(nc2) != float: # проверка на NaN
    train_dict["input"].append(input_str)
    train_dict["target"].append(str(nc2))
  if type(nc3) != float: # проверка на NaN
    train_dict["input"].append(input_str)
    train_dict["target"].append(str(nc3))

train_df = pd.DataFrame(train_dict)

# перемешиваем датасет; задаём seed, чтобы результаты перемешивания совпадали между разными запусками
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# и всё, этого достаточно! токенизаторы у каждой модели свои, лемматизация и другая нормализация
# только уменьшат количество полезных данных для transformer-нейросетей

# пример того, что у нас в датафрейме:
train_df.sample(10)

,input,target
11981,и вот какая скотина это постит? какие лайки? г...,и вот какой плохой человепк это постит? какие ...
1853,ебнутая!!!! выкинул бы её и все дела!!!!,выкинул бы её и все дела!!!!
3818,"можешь не говорить какое, это пердёж таких как...","можешь не говорить какое, это неправда"
3417,а где папаша????? и зачем от таких уродов рожа...,а где папаша????? и зачем от таких людей рожат...
665,"так всех Элиенс лишитесь,неблагодарные говнюки(","так всех Элиенс лишитесь,неблагодарные люди("
204,блять! так вот кто разбрасывает коронавирус!:-...,Так вот кто разбрасывает коронавирус!
3347,валить таких мразей сразу с порога!!!!!!!,Наказывать таких сразу с порога
9988,"Пидоры, а я так надеелся, что меня дропнут(","а я так надеелся, что меня дропнут("
7242,"мышка пидараска глючит,невозможно играть!(((","Мышка не работает, невозмодно игратт"
7667,сейчас эту мразь будем содержать. они не работ...,Сейчас этого человека будем содержать. Они не ...


## Train mBART (FB)

In [ ]:
# устанавливаем необходимые библиотеки
!pip install -qqq happytransformer sentencepiece

     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 17.3 MB/s 
     |████████████████████████████████| 325 kB 50.4 MB/s 
     |████████████████████████████████| 3.8 MB 42.6 MB/s 
     |████████████████████████████████| 67 kB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 212 kB 52.5 MB/s 
     |████████████████████████████████| 134 kB 57.5 MB/s 
     |████████████████████████████████| 127 kB 50.4 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 6.5 MB 62.7 MB/s 
     |████████████████████████████████| 895 kB 58.8 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 54.0 MB/s 
     |████████████████████████████████| 271 kB 75.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
import csv

# подготавливаем обучающие данные
train_df.to_csv("train.csv", index=False, quoting=csv.QUOTE_ALL)

In [ ]:
# dirty fix: нам нужен правильный токенизатор для multilingual модели
import os

broken_path = !python -c "import os; import happytransformer; print(os.path.dirname(happytransformer.__file__))"
broken_path = str(list(broken_path)[0])

fix_file = []
with open(os.path.join(broken_path, "happy_transformer.py"), "r") as in_file:
    for line in in_file.read().split("\n"):
        if line == "            self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=use_auth_token)":
            fix_file.append("            self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=use_auth_token, src_lang='ru_RU', tgt_lang='ru_RU')")
        else:
            fix_file.append(line)
with open(os.path.join(broken_path, "happy_transformer.py"), "w") as out_file:
    out_file.write("\n".join(fix_file))

In [ ]:
from happytransformer import HappyTextToText, TTTrainArgs
from transformers import AutoTokenizer

# берём модель mBART от FB, размер чекпойнта large
# cc25 содержит в себе меньше всего языков, что повышает точность модели
model = HappyTextToText("BART", "facebook/mbart-large-cc25")

# 3 эпохи обучения
args = TTTrainArgs(num_train_epochs=3) 
model.train("train.csv", args=args)

03/20/2022 18:22:47 - INFO - happytransformer.happy_transformer -   Using model: cuda
03/20/2022 18:22:49 - INFO - happytransformer.happy_transformer -   Preprocessing training data...



Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-805d7d54be52421d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


03/20/2022 18:22:54 - INFO - happytransformer.happy_transformer -   Training...
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12206
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 36618


Step,Training Loss
500,5.651600
1000,3.454500
1500,2.974500
2000,3.002700
2500,2.569600
3000,2.093800
3500,1.945400
4000,1.824500
4500,1.760500
5000,1.759700




Training completed. Do not forget to share your model on huggingface.co/models =)




## Evaluate model

In [ ]:
from tqdm.notebook import tqdm

# подготавливаем тест сет
test_df = pd.read_csv("./russe_detox_2022/data/input/test.tsv", sep="\t")
output = [model.generate_text(entry) for entry in tqdm(list(test_df["toxic_comment"]))]
output[:20]

[TextToTextResult(text='кто эту придумку придумывает.'),
 TextToTextResult(text='В такой в такой ситуации виноваты , из Ростелекома, у которых даже кошка может купить фильм с пульта'),
 TextToTextResult(text='актёр может и не плохой, но как человек - плохой'),
 TextToTextResult(text='мочите всех кто нарушает общественный порядок'),
 TextToTextResult(text='такие же люди и привели этих людей.'),
 TextToTextResult(text='А зачем тогда ты здесь это писал?.'),
 TextToTextResult(text='главный неудачник года уханя повар из полицейский из миннеаполиса сварщик из бейрута президент минска из d'),
 TextToTextResult(text='Начни со сваих людей..('),
 TextToTextResult(text='дайте уже пожить создать семью отдал 35 лет жизни кормил власти 30 лет мой отец 90 лет хватит'),
 TextToTextResult(text='а ты,что 41 год помнишь? сколько этой женщине денег заплатили,чтоб она такую чу несла.'),
 TextToTextResult(text='С которым через час расстанешься и будешь с другими общаться?'),
 TextToTextResult(text='Утые люд

In [ ]:
# сохраняем предсказания, подготавливаем для кодалаба
with open("output_bart_large_3.txt", "w") as out_file:
    out_file.write("\n".join([entry.text for entry in output]))

!zip output_bart_large_3.zip output_bart_large_3.txt

## Export model

In [ ]:
# сохраняем модель
model.save("bart_large_3/")
!tar -czf bart_large_3.tar.gz bart_large_3

Configuration saved in bart_large_3/config.json
Model weights saved in bart_large_3/pytorch_model.bin
tokenizer config file saved in bart_large_3/tokenizer_config.json
Special tokens file saved in bart_large_3/special_tokens_map.json


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# выгружаем веса на гугл диск

#!mkdir /content/drive/MyDrive/rudetox
!cp bart_large_3.tar.gz /content/drive/MyDrive/rudetox
!cp output_bart_large_3.zip /content/drive/MyDrive/rudetox